# World Life Expectancy - SQL Project

This project analyzes global life expectancy data using SQL.  
The aim is to explore how health and economic factors influence life expectancy across countries from 2000 to 2015.  
Data was cleaned, missing values handled, and queries were written to calculate averages, detect trends, and compare countries by status (developed vs. developing).  
The analysis highlights key differences in life expectancy and reveals regions with notable improvements over time.

## Table of Contents
* [1. Data Preview](#data-preview)
* [2. Cleaning Data](#cleaning-data)
   - [2.1 Duplicates](#duplicates)
   - [2.2 Missing Values](#missing-values)
* [3. Analysis](#analysis)
   - [3.1. Correlation between GDP and life expectancy](#GDP-CORR)
   - [3.2 Correlation between Status of a country and its Life Expectancy](##32)
   - [3.3 BMI vs. Life Expectancy Analysis](##33)
* [4. Conclusions](#conclusions)


## 1. Data Preview <a id="data-preview"></a>

In [6]:
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [7]:
csv_name = "WorldLifeExpectancy.csv"
# Load csv
df = pd.read_csv(csv_name)
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,percentage expenditure,Measles,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,GDP,thinness 1-19 years,thinness 5-9 years,Schooling,Row_ID
0,Afghanistan,2022,Developing,65.0,263,62,71.3,1154,19.1,83,6,65,0.1,584,17.2,17.3,10.1,1
1,Afghanistan,2021,Developing,59.9,271,64,73.5,492,18.6,86,58,62,0.1,613,17.5,17.5,10.0,2
2,Afghanistan,2020,Developing,59.9,268,66,73.2,430,18.1,89,62,64,0.1,632,17.7,17.7,9.9,3
3,Afghanistan,2019,Developing,59.5,272,69,78.2,2787,17.6,93,67,67,0.1,670,17.9,18.0,9.8,4
4,Afghanistan,2018,Developing,NaN,275,71,7.1,3013,17.2,97,68,68,0.1,64,18.2,18.2,9.5,5


## 2. Cleaning Data<a id="cleaning-data"></a>

### 2.1 Duplicates <a id="duplicates"></a>
Each country should have 1 record per year, hence each country-year pair should appear only once in the dataset.     
**Steps I will take:**
- Identify duplicates  
- Create a backup and delete them  
- Verify the results
  
**Identifying**  
I began by checking for duplicate combinations using a `COUNT(*)` aggregation.

<img src="cleaning/cleaning_img_1.jpeg" alt="Cleaning Data Screenshot" width="900">

>output:

<img src="cleaning/cleaning_img_2.jpeg" alt="Cleaning Data Screenshot" width="1000">
3 countries have double records of a certain year.

**Deleting (after creating backup table)**  
After identifying that 3 countries had duplicate year records, I used a subquery with a `ROW_NUMBER()` window function to remove the extra rows, identified by their Row_ID:

<img src="cleaning/cleaning_img_5.jpeg" alt="Cleaning Data Screenshot" width="1000">

**Verification Step**  
After deletion, I re-ran a check to confirm each country has only one record per year.
<img src="cleaning/cleaning_img_6.jpeg" alt="Cleaning Data Screenshot" width="900">
No rows were returned, confirming duplicates were removed successfully.

### 2.2 Handling Missing Values <a id="missing-values"></a>
**2.2.1 Status Field**  
    
First, I filled in the missing values in the **Status** column.   
I noticed that each country only has one status; either Developed or Developing.   
So if a country had a missing value, I could safely fill it in based on its known status.

#### Recognize missing values:
<img src="cleaning/cleaning_img_7.jpeg" alt="Cleaning Data Screenshot" width="900">

#### Checking all possible values:

<img src="cleaning/cleaning_img_8.jpeg" alt="Cleaning Data Screenshot" width="700">

#### Checking each country has only one status:
This query counts distinct Status values (Developed or Developing) for each country where the status is not null, helping verify that every country is classified as only one (not counting missing values).

<img src="cleaning/cleaning_img_9.jpeg" alt="Cleaning Data Screenshot" width="900">

Output showed no rows, verifying ach country has 1 status.

#### Filling missing values
This query updates missing values in the **Status** column by using a self-join on the `Country` field.   
It matches records with blank status `(t1.Status = "")` to other rows of the same country `(t2)` that already have a defined   
status `(t2.Status = "Developing")`. When the matched record’s status is Developing, it fills the blank value with Developing.   
I then did the same with "Developed".
<img src="cleaning/cleaning_img_10.jpeg" alt="Cleaning Data Screenshot" width="2000">


**2.2.2 Life Expectancy Field**
  
To evaluate life expectancy trends by country, I first identified missing entries in the dataset:

In [8]:
## MISSING VALUES (I used pandas here to shorten the report):
df[df["Life expectancy "].isnull()][["Country", "Year", "Life expectancy "]]

,Country,Year,Life expectancy
4,Afghanistan,2018,NaN
20,Albania,2018,NaN


To fill in I joined the table to itself twice, once with the year before (t2) and once with the year after (t3), and replaced missing values with the **average of the two adjacent years**. This method is known as **linear interpolation**, as it estimates values by assuming a straight-line trend between two known data points.

<img src="cleaning/cleaning_img_14.jpeg" alt="Cleaning Data Screenshot" width="2000">

After updating, I compared the average life expectancy before and after filling:

<img src="cleaning/cleaning_img_15.jpeg" alt="Cleaning Data Screenshot" width="2000">

Since the average decreased, those new values must generally be below the original mean.
This suggests that missing life expectancy entries likely belonged to countries or years with lower expected life expectancy than the rest of the dataset, which we will check in the next section.

## 3. Exploratory Data Analysis <a id="analysis"></a>

### 3.1. Correlation between GDP and life expectancy
  
I would like to check does higher GDP correlates with higher life expectancy?
To do so, I:
1. Checked the `AVG(Life_expectancy`, and `AVG(GDP)`
2. **Using CASE function** for seeing what is the AVG(Life_expectancy) for all entries above and below the AVG`AVG(GDP)`

<img src="exploratory/IMG_1.jpeg" alt="Exploratory Data Screenshot" width="2000">

| Results show that countries with higher GDP, higher than the overall avarage, had significant higher life expectancy, from the total avarage and also from countries with lower GDP.

### 3.2 Correlation between Status of a country and its Life Expectancy


I would like to see, how many countries are in status developed or developing, and is there a correlation between a coutry's status and its life expectancy?

<img src="exploratory/IMG_2.jpeg" alt="Exploratory Data Screenshot" width="2000">

>Clearly, more countries are Developing, which can affects the life expectancy to be closer to the overall AVG (Law of large numbers). Developed countries, when compared, has longer life expectancy on avarage.

### 3.3 BMI vs. Life Expectancy Analysis

To understand how average BMI relates to overall health outcomes around the world, I aggregated life-expectancy and BMI data for every country in the world_life_expectancy dataset. For each country, I calculated:
* Average Life Expectancy (rounded to 1 decimal)  
* Average BMI (rounded to 1 decimal)
  
I removed invalid values by keeping only countries with positive averages, then ranked them from lowest to highest BMI.
Here is the query I used:


<img src="exploratory/IMG_3.jpeg" alt="Exploratory Data Screenshot" width="2000">

> What This Shows:
This ranking highlights which countries have the lowest average BMI, allowing patterns to emerge between nutritional trends and life expectancy. Countries with lower BMI values (lower than what is considered healthy) often means regions with different socioeconomic or dietary conditions, and comparing these values to life expectancy helps reveal broader public-health insights.